In [ ]:
from __future__ import division
from pyoperators import pcg
from pysimulators import profile
from pylab import *

# QUBIC packages
import qubic
from qubicpack.utilities import Qubic_DataDir
from qubic.data import PATH
from qubic.io import read_map
from qubic import QubicSkySim as qss
import sys
sys.path.append('/Users/mregnier/Desktop/PhD Regnier/mypackages')


import Acquisition as Acq

# Display packages
import healpy as hp
import matplotlib.pyplot as plt

# FG-Buster packages
import component_model as c
import mixing_matrix as mm

# General packages
import numpy as np
import pysm3
import warnings
from qubic import QubicSkySim as qss
import pysm3.units as u
from importlib import reload
from pysm3 import utils
import myPCG as mypcg

from qubic import SpectroImLib as sp
from importlib import reload
import gc
import copy
from pyoperators import *


# PyOperators packages
from pyoperators import (
    BlockColumnOperator, BlockDiagonalOperator, BlockRowOperator,
    CompositionOperator, DiagonalOperator, I, IdentityOperator,
    MPIDistributionIdentityOperator, MPI, proxy_group, ReshapeOperator,
    rule_manager, pcg, Operator)

from pysimulators.interfaces.healpy import HealpixConvolutionGaussianOperator
warnings.filterwarnings("ignore")
# %matplotlib inline

## Now QUBIC and Planck acquisitions used for reconstruction

# # %config InlineBackend.figure_format='retina'
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
reload(Acq)
nside = 256
relative_bandwidth = 0.25
band = 220
band_planck = 217
seed = 42
noiseless = False
effective_duration = 3e0
rwplanck = 1.
tol = 1e-40
noise_planck_level = 0
noise_qubic_level = 0


npointings = 3000
Nf_TOD = 20
Nf_recon = 1
fact_sub = 10
maxiter = 30
factor_nside_tod = 1

reload(Acq)

# Repository for dictionary
global_dir = Qubic_DataDir()
print(global_dir)
dictfilename = global_dir + 'dicts/pipeline_demo.dict'

# Read dictionary chosen
d_TOD = qubic.qubicdict.qubicDict()
d_TOD.read_from_file(dictfilename)

d_TOD['nf_recon'] = Nf_TOD
d_TOD['nf_sub'] = Nf_TOD
d_TOD['nside'] = nside * factor_nside_tod
npix=12*d_TOD['nside']**2
d_TOD['RA_center'] = 0
d_TOD['DEC_center'] = -57
center = qubic.equ2gal(d_TOD['RA_center'], d_TOD['DEC_center'])
d_TOD['effective_duration'] = effective_duration
d_TOD['npointings'] = npointings
d_TOD['tol'] = 5e-4
d_TOD['filter_nu'] = band * 1e9
d_TOD['photon_noise'] = False#not noiseless
d_TOD['noiseless'] = noiseless
d_TOD['config'] = 'FI'
d_TOD['MultiBand'] = True
d_TOD['planck'] = True
d_TOD['dtheta'] = 15
#d_TOD['synthbeam_dtype'] = float

print('*************** Noise **************')
print('Noisless:      {}'.format(d_TOD['noiseless']))
print('Photon Noise:  {}'.format(d_TOD['photon_noise']))
print('************************************')

sky_config = {'cmb':42}
#sky_config = {'dust':'d0'}
#sky_config = {'cmb':3, 'dust':'d0'}

reload(Acq)
qubic_acquisition = Acq.QubicIntegrated(d_TOD, Nsub=Nf_TOD, Nrec=Nf_TOD)




# QUBIC TOD

In [ ]:
# Create Nsub maps
m_sub_tod = qubic_acquisition.get_PySM_maps(sky_config, qubic_acquisition.allnus)
qubic_acquisition = Acq.QubicIntegrated(d_TOD, Nsub=Nf_TOD, Nrec=Nf_TOD)

# Create operator
H_TOD = qubic_acquisition.get_operator(convolution=True, myfwhm=None, fixed_data=None)
n_qubic = qubic_acquisition.get_noise()

# Create TOD
if Nf_recon == 1 and fact_sub == 1:
    TOD_QUBIC = H_TOD(m_sub_tod[0]).ravel() + (n_qubic.ravel() * noise_qubic_level)
else:
    TOD_QUBIC = H_TOD(m_sub_tod).ravel() + (n_qubic.ravel() * noise_qubic_level)

    
### Now put back qubic_acqisition to normal nside 
d_TOD['nside'] = nside
qubic_acquisition = Acq.QubicIntegrated(d_TOD, Nsub=Nf_TOD, Nrec=Nf_TOD)
cov = qubic_acquisition.get_coverage()
covnorm = cov/cov.max()
threshold = 0
#seenpix = generate_circular_mask(256, 10, center[0], center[1])
seenpix = covnorm > threshold
notseenpix = covnorm < threshold


In [ ]:
plt.figure(figsize=(15, 5))
for k in range(3):
    hp.gnomview(m_sub_tod[0,:,k], rot=center, reso=5, cmap='jet', sub=(1,3,k+1))

# Planck TOD

In [ ]:
def integration(nu_min, nu_max, Nintegr, sky_config, d, fwhm_subbands = None, factor_nside_tod=1):
    print(f'Integration from {nu_min:.2f} to {nu_max:.2f} GHz with {Nintegr} steps')
    obj = Acq.QubicIntegrated(d, Nsub=Nintegr, Nrec=Nintegr)
    if Nintegr == 1:
        allnus = np.array([np.mean([nu_min, nu_max])])
    else:
        allnus = np.linspace(nu_min, nu_max, Nintegr)
    m = obj.get_PySM_maps(sky_config, nus=allnus)
    
    if fwhm_subbands is not None:
        for i in range(Nintegr):
            C = HealpixConvolutionGaussianOperator(fwhm=fwhm_subbands[i])
            m[i] = C(m[i])
    
    return np.array([np.mean(m, axis=0)])


In [ ]:
planck_acquisition = Acq.PlanckAcquisition(band_planck, qubic_acquisition.scene)

d_formaps = copy.deepcopy(d_TOD)
d_formaps['nside'] = nside
d_formaps['nf_recon'] = Nf_recon
d_formaps['nf_sub'] = Nf_recon * fact_sub

### We need to degrade High Nside maps used for TOD to the "normal" nside that will be used for Planck
m_sub = np.zeros((Nf_TOD, 12*nside**2, 3))
for k in range(3):
    for i in range(Nf_TOD):
        m_sub[i,:,k] = hp.ud_grade(m_sub_tod[i,:,k], nside)


#planck_obs = integration(band_planck-35, band_planck+35, qubic_acquisition.Nsub, sky_config, d_formaps, fwhm_subbands=None) # Planck map
n_planck = planck_acquisition.get_noise()  # Planck noise

m_planck_for_tod = np.zeros((Nf_recon, 12*nside**2, 3))
m_planck_all = np.zeros((Nf_recon * fact_sub, 12*nside**2, 3))

# Convolving Planck map to QUBIC resolutions
print('Planck maps for TOD')
for i in range(Nf_recon):
    
    target = np.min(qubic_acquisition.allfwhm[i*fact_sub:(i+1)*fact_sub])
    #target = allfwhm_ref[i].copy()                   # Target FWHM
    print(i,target)
    C = HealpixConvolutionGaussianOperator(fwhm=target)                                     # Operator
    m_planck_for_tod[i] = C(m_sub[i] + n_planck.copy() * noise_planck_level)                # Convolution

print('All Planck maps')
for i in range(Nf_recon*fact_sub):
    
    target = qubic_acquisition.allfwhm[i]                   # Target FWHM
    print(i,target)
    C = HealpixConvolutionGaussianOperator(fwhm=target)                                     # Operator
    m_planck_all[i] = C(m_sub[i] + n_planck.copy() * noise_planck_level)                    # Convolution
    
plt.figure(figsize=(15, 5))
stk = ['I', 'Q', 'U']
j=0
for i in range(Nf_recon):
    for k in range(3):
        if k == 0:
            r = 200
        else:
            r = 12
        hp.mollview(m_planck_for_tod[i, :, k], cmap='jet', sub=(Nf_recon, 3, j+1), min=-r, max=r, title=f'{stk[k]} - FWHM = {np.mean(qubic_acquisition.allfwhm[i*fact_sub:(i+1)*fact_sub]):.4f} rad')
        j+=1

plt.show()

R = ReshapeOperator(m_planck_for_tod.shape, (m_planck_for_tod.shape[0]*m_planck_for_tod.shape[1]*m_planck_for_tod.shape[2]))
TOD_PLANCK = R(m_planck_for_tod)

In [ ]:
hp.mollview(m_sub_tod[0,:,0], cmap='jet', title='m_sub_tod: nside={}'.format(hp.npix2nside(len(m_sub_tod[0,:,0]))))
hp.mollview(m_sub[0,:,0], cmap='jet', title='m_sub: nside={}'.format(hp.npix2nside(len(m_sub[0,:,0]))))

# Final TOD

In [ ]:
print(TOD_QUBIC)
print(TOD_PLANCK)

In [ ]:
tod = np.r_[TOD_QUBIC, TOD_PLANCK]

plt.figure(figsize=(15, 10))
plt.subplot(2, 1, 1)
plt.plot(TOD_QUBIC/TOD_QUBIC.max())

plt.subplot(2, 1, 2)
plt.plot(TOD_PLANCK/TOD_PLANCK.max())

plt.show()

In [ ]:
qubicplanck_acquisition = Acq.QubicPlanckMultiBandAcquisition(qubic_acquisition, planck_acquisition)

myfwhm = np.array([])
for i in range(Nf_recon):
    myfwhm = np.append(myfwhm, np.sqrt(qubic_acquisition.allfwhm[i*fact_sub:(i+1)*fact_sub]**2 - np.min(qubic_acquisition.allfwhm[i*fact_sub:(i+1)*fact_sub])**2))

print(f'Reconstruction using FWHM : {myfwhm} [rad]')


In [ ]:
d_formaps = copy.deepcopy(d_TOD)
d_formaps['nside'] = nside
d_formaps['nf_recon'] = Nf_recon
d_formaps['nf_sub'] = Nf_recon * fact_sub

# QUBIC acqusition
qubic_acquisition = Acq.QubicIntegrated(d_formaps, Nsub=Nf_recon * fact_sub, Nrec=Nf_recon)

# Joint acquisition
qubicplanck_acquisition = Acq.QubicPlanckMultiBandAcquisition(qubic_acquisition, planck_acquisition)
R = ReshapeOperator((1, 786432, 3), (786432, 3))

user = 'Pierre'
M = Acq.get_preconditioner(np.ones(12*nside**2))
if user == 'full':
    
    H = qubicplanck_acquisition.get_operator(convolution=True, myfwhm=myfwhm, fixed_data=None)
    invN = qubicplanck_acquisition.get_invntt_operator(beam_correction=None, seenpix=None, mask=None)

    with rule_manager(none=True):
        if Nf_recon == 1:
            A = R.T * (H.T * invN * H) * R
            b = R.T * (H.T * invN * tod)
        else:
            A = (H.T * invN * H)
            b = (H.T * invN * tod)
        
elif user == 'Pierre':
    H = qubicplanck_acquisition.get_operator(convolution=True, myfwhm=myfwhm, fixed_data=None)
    invN = qubicplanck_acquisition.get_invntt_operator(beam_correction=None, seenpix=None, mask=None)
    
    ### Solve PCG
    U = (
        ReshapeOperator((Nf_recon * sum(seenpix) * 3), (Nf_recon, sum(seenpix), 3)) *
        PackOperator(np.broadcast_to(seenpix[None, :, None], (Nf_recon, seenpix.size, 3)).copy())
    ).T

    with rule_manager(none=True):
        if Nf_recon == 1:
            A = U.T * R.T * H.T * invN * H * R * U
            x_planck = m_planck_for_tod * (1 - seenpix[None, :, None])
            b = U.T ( R.T * H.T * invN * (tod - H(R(x_planck))))
        else:
            A = U.T * H.T * invN * H * U
            x_planck = m_planck_for_tod * (1 - seenpix[None, :, None])
            b = U.T (  H.T * invN * (tod - H(x_planck)))
    
    
    
elif user == 'Mathias':
    fixed_data = m_planck_all.copy()
    fixed_data[:, seenpix, :] = 0
    H = qubicplanck_acquisition.get_operator(convolution=False, fixed_data=fixed_data)
    invN = qubicplanck_acquisition.get_invntt_operator(beam_correction=None, seenpix=None, mask=None)
    
    A = H.T * invN * H
    b = H.T * invN * tod
elif user == 'old':
    maskinit = np.array(seenpix, dtype=float)
    index_0 = maskinit == 0
    maskinit[index_0] += 1e10

    H = qubicplanck_acquisition.get_operator(convolution=False, fixed_data=None)
    invN = qubicplanck_acquisition.get_invntt_operator(beam_correction=None, seenpix=None, mask=None)
    
    A = H.T * invN * H
    b = H.T * invN * tod
    
    #for i in range(1, Nf_recon+1):
    #    invN.operands[i].operands[1] /= DiagonalOperator(np.array([maskinit, maskinit, maskinit]).T)
else:
    raise TypeError('We would like to have more people..')

In [ ]:
U.shapeout

In [ ]:
if user == 'Pierre' or user == 'Mathias':
    m_ini = m_planck_for_tod.copy()
    solution_qubic_planck = mypcg.pcg(A, b, x0=m_planck_for_tod[:, seenpix], M=M, tol=tol, disp=True, maxiter=maxiter)
elif user == 'full':
    solution_qubic_planck = mypcg.pcg(A, b, x0=m_planck_for_tod, M=M, tol=tol, disp=True, maxiter=maxiter)
elif user == 'old':
    solution_qubic_planck = mypcg.pcg(A, b, x0=m_planck_for_tod[0], M=M, tol=tol, disp=True, maxiter=maxiter)
else:
    raise TypeError('We would like to have more people..')

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(solution_qubic_planck['x']['convergence'])
plt.yscale('log')
plt.show()

In [ ]:
print(solution_qubic_planck['x']['x'])

In [ ]:
mysolution = m_planck_for_tod.copy()
if user == 'Pierre' or user == 'Mathias':
    mysolution[:, seenpix] = solution_qubic_planck['x']['x'].copy()
elif user == 'old' or user == 'full':
    mysolution = solution_qubic_planck['x']['x'].copy()
else:
    raise TypeError('We would like to have more people..')
    
print(mysolution)
print(np.mean(mysolution))

In [ ]:
print('mysolution', np.shape(mysolution))
print('m_planck_all', np.shape(m_planck_all))
print('m_planck_for_tod', np.shape(m_planck_for_tod))
print('m_sub', np.shape(m_sub))

In [ ]:
myinputs = m_planck_for_tod.copy()

In [ ]:
res=25
istk=0
minr=10


for k in range(Nf_recon):
    plt.figure(figsize=(20, 8))
    suptitle('Sub-band {}'.format(k), fontsize=20, y=1.05)
    for i in range(3):
        if i == 0:
            r = 1
            v = 300
        else:
            r = 0.1
            v = 8
        hp.gnomview(mysolution[k, :, i], cmap='jet', rot=center, reso=res, min=-v, max=v, sub=(3, 3, 1+3*i), title='Output sub {}'.format(k))
        hp.gnomview(myinputs[k,:, i], cmap='jet', rot=center, reso=res, min=-v, max=v, sub=(3, 3, 2+3*i), title='Input sub {}'.format(k))

        hp.gnomview(mysolution[k, :, i]-myinputs[k,:, i], cmap='jet', rot=center, reso=res, title='Residual sub {}'.format(k),
                                min=-r, max=r, sub=(3, 3, 3+3*i))
    tight_layout()
    
#plt.show()

In [ ]:
import qubic.fibtools as ft
covnorm = cov/np.max(cov)
uvpix = np.array(hp.pix2vec(hp.npix2nside(len(cov)), np.arange(len(cov))))
uvcenter = np.array(hp.pix2vec(hp.npix2nside(len(cov)), np.argmax(cov)))
dcenter = np.degrees(np.arccos(np.dot(uvpix.T, uvcenter)))

plt.figure(figsize=(20, 8))
for i in range(3):
    for k in range(Nf_recon):
        new_res0 = mysolution[k, :, i]-myinputs[k,:, i]
        x0,y0,sx0,sy0,_ = ft.profile(dcenter, new_res0, nbins=20, rng=[0,55], plot=False)

        plt.subplot(3, 2, i*2+1)
        plt.errorbar(x0, y0, yerr=sy0, fmt='o', capsize=3, label='Sub={}'.format(k))
        plt.axhline(0, color='black')
        plt.xlabel('Radius [deg]')
        plt.ylabel('Mean')
        plt.subplot(3, 2, i*2+2)
        plt.plot(x0, sy0,'o-', label='Sub={}'.format(k))
        plt.xlabel('Radius [deg]')
        plt.ylabel('RMS Residuals')
        plt.axhline(0, color='black')
    plt.legend()
tight_layout()